In [ ]:
import blackjack_original as jack
import blackjack_double as jack_double

import numpy as np
from tqdm.notebook import tqdm

#### Наивная стратегия: делать stand при сумме карт 16-21, иначе hit

In [ ]:
# 0 - stand
# 1 - hit
env = jack.BlackjackEnv(natural=True)

episods = 100000
state_stand = [16, 17, 18, 19, 20, 21]
reward_average = 0
step_average = 0

for i in tqdm(range(episods),desc="TRAINING"):
    player, dealer, usable_ace = env.reset()
    done = False
    step=1
    while(done==False):
        step+=1
        if player in state_stand:
            (player, dealer, usable_ace), reward, done, _ = env.step(0)            
        else:
            (player, dealer, usable_ace), reward, done, _ = env.step(1)
    reward_average += reward
    step_average += step
    
reward_average = reward_average/episods
step_average = step_average/episods

print(f"Наивная стратегия делать stand при сумме {state_stand}\nСредний доход:{reward_average}\nСреднее кол-во ходов:{step_average}")

#### Off-policy TD control "Q learning"

In [ ]:
class TDcontrol():
    def __init__(self, gym_env, epsilon, alpha, gamma):
        self.env = gym_env
        
        A_count = self.env.action_space.n
        S_shape = []
        for space in self.env.observation_space:
            S_shape.append(space.n)
        shape = S_shape
        shape.append(A_count)
        self.Q = np.zeros((shape))
        
        self.epsilon = epsilon
        self.alpha = alpha
        self.gamma = gamma        
            
    def set(self,):
        pass
    
    def train(self, episods):
        for _ in tqdm(range(episods),desc="TRAINING"):
            state = self.env.reset()
            done = False
            while not(done):
                action = self.__get_action(state)
                state_new, reward, done, _ = self.env.step(action)
                self.Q[state][action] = self.Q[state][action] + self.alpha*(reward + self.gamma*np.max(self.Q[state_new]) - self.Q[state][action])
                state = state_new
    
    def game(self, episods):
        reward_average = 0
        step_average = 0
        for _ in tqdm(range(episods),desc="GAMING"):
            state = self.env.reset()
            done = False
            step = 1
            while not(done):
                action = self.pi(state)
                state, reward, done, _ = self.env.step(action)
                step+=1
            reward_average+=reward
            step_average+=step
            
        return reward_average/episods, step_average/episods
                
    def pi(self, state):
        return np.argmax(self.Q[state])
        
    def __get_action(self, state):
        if np.random.uniform(0,1)<self.epsilon:
            return self.env.action_space.sample()
        else:
            return np.argmax(self.Q[state])
        
    def __retr(self,player, dealer, usable_ace):
        pass

In [ ]:
env = jack.BlackjackEnv(natural=True)

alphas = [1e-6, 1e-5, 1e-4]
gammas = [0.5, 0.8, 0.9, 1]
epsilons = [0.8, 0.85, 0.9, 0.95]
episods_count = 200000
results = []

for alpha in alphas:
    for gamma in gammas:
        for epsilon in epsilons:
            qmodel = TDcontrol(env, epsilon, alpha, gamma)
            qmodel.train(episods_count)
            rew,step = qmodel.game(episods_count)
            result = f"{rew} steps average={step} alpha={alpha} gamma={gamma} epsilon={epsilon}"
            results.append(result)
            print("Off-policy TD control (Q learning)")
            print(result)

#### BlackJack with Double

In [ ]:
env = jack_double.BlackjackEnv(natural=True)

alphas = [1e-6, 1e-5, 1e-4]
gammas = [0.5, 0.8, 0.9, 1]
epsilons = [0.8, 0.85, 0.9, 0.95]
episods_count = 200000
results = []

for alpha in alphas:
    for gamma in gammas:
        for epsilon in epsilons:
            qmodel = TDcontrol(env, epsilon, alpha, gamma)
            qmodel.train(episods_count)
            rew,step = qmodel.game(episods_count)
            result = f"{rew} steps average={step} alpha={alpha} gamma={gamma} epsilon={epsilon}"
            results.append(result)
            print("Off-policy TD control (Q learning)")
            print(result)

In [ ]:
deck = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 10, 10, 10]*4


In [ ]:
deck.remove(10)
len(deck)

In [ ]:
env._get_obs()

In [ ]:
arr = np.zeros((2,3,4))
arr.

In [ ]:
arr = np.random.randint(0,10,size=2*3*4).reshape(2,3,4)
arr

In [ ]:
state = (0,int(True))
arr[state][0]=400
arr

In [ ]:
def cmp(a, b):
    return float(a > b) - float(a < b)

cmp(12, 13)*2